In [101]:
import pandas as pd
import re
import numpy as np
import os

In [102]:
input_folder = "./data/AL"
data_path = "./data/"

In [103]:
for root, dirs, files in os.walk(input_folder):
    print(files)

[]
[]
[]
['NIBRS_PROP_DESC_TYPE.csv', 'NIBRS_WEAPON_TYPE.csv', 'NIBRS_ASSIGNMENT_TYPE.csv', 'REF_RACE.csv', 'NIBRS_ARREST_TYPE.csv', 'NIBRS_SUSPECTED_DRUG.csv', 'postgres_load.sql', 'NIBRS_AGE.csv', 'NIBRS_INJURY.csv', 'NIBRS_ARRESTEE.csv', 'NIBRS_VICTIM_INJURY.csv', 'NIBRS_PROP_LOSS_TYPE.csv', 'NIBRS_ETHNICITY.csv', 'NIBRS_PROPERTY_DESC.csv', 'NIBRS_JUSTIFIABLE_FORCE.csv', 'NIBRS_SUSPECTED_DRUG_TYPE.csv', 'NIBRS_RELATIONSHIP.csv', 'NIBRS_BIAS_MOTIVATION.csv', 'agencies.csv', 'NIBRS_USING_LIST.csv', 'NIBRS_CRIMINAL_ACT.csv', 'NIBRS_month.csv', 'postgres_setup.sql', 'NIBRS_OFFENSE_TYPE.csv', 'NIBRS_WEAPON.csv', 'NIBRS_VICTIM_TYPE.csv', 'NIBRS_BIAS_LIST.csv', 'README.md', 'NIBRS_incident.csv', 'NIBRS_DRUG_MEASURE_TYPE.csv', 'nibrs_diagram.pdf', 'REF_STATE.csv', 'NIBRS_PROPERTY.csv', 'NIBRS_VICTIM_CIRCUMSTANCES.csv', 'NIBRS_ACTIVITY_TYPE.csv', 'NIBRS_ARRESTEE_GROUPB.csv', 'NIBRS_CIRCUMSTANCES.csv', 'NIBRS_CLEARED_EXCEPT.csv', 'NIBRS_OFFENSE.csv', 'NIBRS_CRIMINAL_ACT_TYPE.csv', 'NIBRS_OFFE

In [104]:
AL_df = pd.DataFrame()
cur_inc = pd.DataFrame()
cur_off = pd.DataFrame()
for root, dirs, files in os.walk(input_folder):
    for f in files:
        if f == "NIBRS_incident.csv" or f == "NIBRS_incident.CSV":
            filepath = root + os.sep + f
            cur_inc = pd.read_csv(filepath)
            cur_inc.columns = cur_inc.columns.str.lower()

        elif f == "NIBRS_OFFENSE.CSV" or f == "NIBRS_OFFENSE.csv":
            filepath = root + os.sep + f
            cur_off = pd.read_csv(filepath)
            cur_off.columns = cur_off.columns.str.lower()
            cur_off.rename(columns={"offense_type_id": "offense_code"}, inplace=True)

        if not cur_inc.empty and not cur_off.empty:
            cur_all = cur_inc.merge(cur_off, on=["data_year", "incident_id"])
            AL_df = pd.concat([AL_df, cur_all], axis=0)
            cur_off, cur_inc = pd.DataFrame(), pd.DataFrame()

In [105]:
print(AL_df["data_year"].value_counts())

data_year
2022    251032
2023    247643
2021    218272
2020     15250
2019      4242
2018      2154
Name: count, dtype: int64


In [106]:
crime_by_location = AL_df

In [107]:
print(
    crime_by_location[["incident_id", "offense_id"]].duplicated().sum(),
    crime_by_location.isnull().sum(),
)

0 data_year                     0
agency_id                     0
incident_id                   0
nibrs_month_id                0
cargo_theft_flag          13299
submission_date               0
incident_date                 0
report_date_flag          20291
incident_hour              3199
cleared_except_id             0
cleared_except_date      670677
incident_status               0
data_home                716947
orig_format                   0
did                           0
offense_id                    0
offense_code                  0
attempt_complete_flag         0
location_id                   0
num_premises_entered     736138
method_entry_code        701536
dtype: int64


In [108]:
crime_by_location["incident_date"] = pd.to_datetime(
    crime_by_location["incident_date"], errors="coerce"
)
crime_by_location["Month"] = crime_by_location["incident_date"].dt.month.astype("Int64")
crime_by_location["Day"] = crime_by_location["incident_date"].dt.day

print(
    crime_by_location[crime_by_location["incident_hour"].isnull()][
        "data_year"
    ].value_counts()
)

data_year
2021    1844
2020    1249
2018      98
2019       8
Name: count, dtype: int64


In [109]:
crime_featured = crime_by_location.copy()
crime_featured = crime_featured.dropna(subset=["incident_hour"])
crime_featured = crime_featured.drop(
    columns=[
        "num_premises_entered",
        "method_entry_code",
        "data_home",
        "cleared_except_date",
    ]
)
crime_featured["WeekDay"] = crime_featured["incident_date"].dt.weekday
crime_featured["IsWeekDay"] = (crime_featured["WeekDay"] < 5).astype(bool)
crime_featured["Isweekend"] = (crime_featured["WeekDay"] >= 5).astype(bool)
hour_bins = [0, 6, 12, 18, 24]
hour_labels = ["Night", "Morning", "Afternoon", "Evening"]
season_bins = [0, 3, 6, 9, 12]
season_labels = ["Winter", "Spring", "Summer", "Fall"]
crime_featured["Season"] = pd.cut(
    crime_featured["Month"], bins=season_bins, labels=season_labels, include_lowest=True
)
crime_featured["TimeCategory"] = pd.cut(
    crime_featured["incident_hour"],
    bins=hour_bins,
    labels=hour_labels,
    include_lowest=True,
)
time_dummies = pd.get_dummies(crime_featured["TimeCategory"])
crime_featured = pd.concat([crime_featured, time_dummies], axis=1)

print(
    crime_featured[crime_featured["TimeCategory"].isnull()]["data_year"].value_counts()
)
print(crime_featured.isnull().sum())

Series([], Name: count, dtype: int64)
data_year                    0
agency_id                    0
incident_id                  0
nibrs_month_id               0
cargo_theft_flag         12388
submission_date              0
incident_date            20291
report_date_flag         20291
incident_hour                0
cleared_except_id            0
incident_status              0
orig_format                  0
did                          0
offense_id                   0
offense_code                 0
attempt_complete_flag        0
location_id                  0
Month                    20291
Day                      20291
WeekDay                  20291
IsWeekDay                    0
Isweekend                    0
Season                   20291
TimeCategory                 0
Night                        0
Morning                      0
Afternoon                    0
Evening                      0
dtype: int64


In [110]:
crime_featured["location_id"].unique()

array([35, 25, 22, 23, 14, 27, 41, 40, 33, 38, 21, 43, 24, 98, 10, 44,  7,
       32, 37, 34, 17, 13, 11, 28, 20,  5,  9, 15, 18, 29, 19, 30, 26, 16,
        1,  8,  6, 39, 31, 12,  2, 36,  4,  3, 42, 45, 46, 47])

In [111]:
def change_id(id):
    if id not in [
        1,
        2,
        4,
        5,
        6,
        7,
        8,
        10,
        11,
        13,
        14,
        16,
        17,
        19,
        20,
        22,
        23,
        24,
        25,
        26,
        27,
        28,
        29,
        30,
        32,
        33,
        35,
        37,
        38,
        41,
        98,
        99,
    ]:
        return 98
    return id


crime_featured["location_id"] = crime_featured["location_id"].apply(change_id)
crime_featured["location_id"].value_counts()
crime_featured["location_id"].unique().size

31

In [112]:
from pandas.tseries.holiday import USFederalHolidayCalendar

cal = USFederalHolidayCalendar()

start_date = crime_featured["incident_date"].min()
end_date = crime_featured["incident_date"].max()

# Get holidays between these dates
holidays = cal.holidays(start=start_date, end=end_date)

# Check if each date is a holiday
crime_featured["IsHoliday"] = (
    crime_featured["incident_date"].dt.date.isin(holidays.date).astype(int)
)

# Time-Series Dataset


In [113]:
import calendar

crime_timed = crime_featured.copy()
crime_timed.rename(columns={"data_year": "Year"}, inplace=True)
crime_timed["crime_count"] = 1
holiday_calendar = (
    crime_timed[crime_timed["IsHoliday"] == 1]
    .drop_duplicates(subset=["Year", "Month", "Day"])  # count each holiday once per day
    .groupby(["Year", "Month"], as_index=False)
    .agg(holiday_num=("Day", "count"))
)

In [114]:
# Group by year and month and calculate aggregate values
aggregated_crime = crime_timed.groupby(
    ["Year", "Month", "location_id"],
    as_index=False,
    observed=False,
).agg(
    crime_count=("crime_count", "sum"),
    weekday=("IsWeekDay", "sum"),
    weekend=("Isweekend", "sum"),
    morning=("Morning", "sum"),
    afternoon=("Afternoon", "sum"),
    evening=("Evening", "sum"),
    night=("Night", "sum"),
)
aggregated_crime = pd.merge(
    aggregated_crime, holiday_calendar, on=["Year", "Month"], how="left"
)

In [ ]:
aggregated_crime["holiday_num"] = aggregated_crime["holiday_num"].fillna(0).astype(int)
aggregated_crime["num_days"] = aggregated_crime.apply(
    lambda row: calendar.monthrange(row["Year"], row["Month"])[1], axis=1
)
ls_features = [
    "morning",
    "afternoon",
    "evening",
    "night",
    # "holiday",
    # "weekday",
    # "weekend",
]
for feature in ls_features:
    aggregated_crime[f"{feature}_rate"] = (
        aggregated_crime[feature] / aggregated_crime["crime_count"]
    )
aggregated_crime["sin_month"] = np.sin(2 * np.pi * (aggregated_crime["Month"] - 1) / 12)
aggregated_crime["cos_month"] = np.cos(2 * np.pi * (aggregated_crime["Month"] - 1) / 12)
# Create a time ID variable for easier shifting
aggregated_crime["time_id"] = aggregated_crime["Year"] * 12 + aggregated_crime["Month"]
print(aggregated_crime.columns)
print(aggregated_crime.isnull().sum())

Index(['Year', 'Month', 'location_id', 'crime_count', 'weekday', 'weekend',
       'morning', 'afternoon', 'evening', 'night', 'holiday_num', 'num_days',
       'morning_rate', 'afternoon_rate', 'evening_rate', 'night_rate',
       'sin_month', 'cos_month', 'time_id'],
      dtype='object')
Year              0
Month             0
location_id       0
crime_count       0
weekday           0
weekend           0
morning           0
afternoon         0
evening           0
night             0
holiday_num       0
num_days          0
morning_rate      0
afternoon_rate    0
evening_rate      0
night_rate        0
sin_month         0
cos_month         0
time_id           0
dtype: int64


In [116]:
aggregated_crime = aggregated_crime.sort_values(["location_id", "Year", "Month"])

In [117]:
crime_pct = aggregated_crime.copy()
crime_pct["crime_pct_change"] = crime_pct["crime_count"].pct_change()
crime_pct.to_csv(f"{data_path}rnn_data_AL.csv", index=False)
crime_pct.sort_values(["Year", "Month", "location_id"])

,Year,Month,location_id,crime_count,weekday,weekend,morning,afternoon,evening,night,holiday_num,num_days,morning_rate,afternoon_rate,evening_rate,night_rate,sin_month,cos_month,time_id,crime_pct_change
0,2021,1,1,8,8,0,1,0,1,6,2,31,0.125000,0.000000,0.125000,0.750000,0.0,1.0,24253,NaN
1,2021,1,2,7,6,1,0,3,1,3,2,31,0.000000,0.428571,0.142857,0.428571,0.0,1.0,24253,-0.695652
2,2021,1,4,3,1,2,2,1,0,0,2,31,0.666667,0.333333,0.000000,0.000000,0.0,1.0,24253,-0.850000
3,2021,1,5,1,1,0,0,0,0,1,2,31,0.000000,0.000000,0.000000,1.000000,0.0,1.0,24253,-0.800000
4,2021,1,6,30,25,5,7,5,3,15,2,31,0.233333,0.166667,0.100000,0.500000,0.0,1.0,24253,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111,2023,12,35,8170,5601,2569,2102,2539,1814,1715,1,31,0.257283,0.310771,0.222032,0.209914,-0.5,0.866025,24288,-0.025757
1112,2023,12,37,282,197,85,64,103,71,44,1,31,0.226950,0.365248,0.251773,0.156028,-0.5,0.866025,24288,0.281818
1113,2023,12,38,50,41,9,21,14,8,7,1,31,0.420000,0.280000,0.160000,0.140000,-0.5,0.866025,24288,-0.358974
1114,2023,12,41,258,184,74,49,79,67,63,1,31,0.189922,0.306202,0.259690,0.244186,-0.5,0.866025,24288,0.023810


In [118]:
df_lag = crime_pct.copy()
for location_group, location_df in df_lag.groupby(["location_id"]):
    # Get indices for this location group
    indices = location_df.index

    # Make sure we're still working with time-ordered data
    temp_df = df_lag.loc[indices].sort_values("time_id")
    all_features = [
        "crime_pct_change",
        # "domestic_rate",
        # "arrest_rate",
        # "holiday_rate",
        "morning_rate",
        "afternoon_rate",
        "evening_rate",
        "night_rate",
        "crime_count",
        # "domestic",
        # "arrest",
        "morning",
        "afternoon",
        "evening",
        "night",
    ]
    # Add lag features
    for lag in [1, 2, 3, 6, 12]:
        for feature in all_features:
            df_lag.loc[indices, f"{feature}_lag{lag}"] = temp_df[feature].shift(lag)

    for lag in [3, 6]:
        for feature in all_features:
            df_lag.loc[indices, f"{feature}_ma{lag}"] = (
                temp_df[feature].rolling(window=lag).mean().shift(1)
            )

# lag_columns = [
#     "crime_count_lag1",
#     "crime_count_lag2",
#     "crime_count_lag3",
#     "crime_count_lag6",
#     "crime_count_lag12",
#     "crime_count_ma3",
#     "crime_count_ma6",
# ]

df_lag = df_lag.dropna()
df_lag.sort_values(["Year", "Month", "location_id"], inplace=True)
df_lag

,Year,Month,location_id,crime_count,weekday,weekend,morning,afternoon,evening,night,...,crime_pct_change_ma6,morning_rate_ma6,afternoon_rate_ma6,evening_rate_ma6,night_rate_ma6,crime_count_ma6,morning_ma6,afternoon_ma6,evening_ma6,night_ma6
373,2022,1,2,14,12,2,7,4,2,1,...,0.344719,0.162943,0.576770,0.106716,0.153571,10.000000,1.833333,5.500000,1.333333,1.333333
374,2022,1,4,8,7,1,1,4,1,2,...,1.566948,0.168340,0.335525,0.474396,0.021739,14.000000,1.833333,3.666667,8.000000,0.500000
375,2022,1,5,11,7,4,4,2,0,5,...,-0.015087,0.201651,0.143196,0.140741,0.514412,11.000000,2.500000,1.333333,1.000000,6.166667
376,2022,1,6,85,68,17,13,25,27,20,...,-0.016226,0.335767,0.338251,0.104511,0.221472,70.166667,23.333333,24.333333,8.166667,14.333333
377,2022,1,7,269,223,46,136,70,9,54,...,-0.037129,0.413014,0.285186,0.028450,0.273350,190.333333,78.166667,54.333333,5.500000,52.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111,2023,12,35,8170,5601,2569,2102,2539,1814,1715,...,-0.033535,0.262992,0.288877,0.228298,0.219833,9540.000000,2506.333333,2746.500000,2184.500000,2102.666667
1112,2023,12,37,282,197,85,64,103,71,44,...,-0.053353,0.223273,0.328167,0.261427,0.187133,302.000000,67.000000,98.833333,79.500000,56.666667
1113,2023,12,38,50,41,9,21,14,8,7,...,0.568019,0.435502,0.298407,0.136416,0.129674,79.166667,31.833333,26.333333,11.000000,10.000000
1114,2023,12,41,258,184,74,49,79,67,63,...,-0.014962,0.224375,0.282149,0.254703,0.238773,289.500000,64.500000,81.166667,74.166667,69.666667


In [119]:
df_complete = df_lag.copy()

In [120]:
df_time_analysis = pd.DataFrame(
    {
        "Column": df_complete.columns,
        "Unique Values": df_complete.nunique().values,
        "Type": df_complete.dtypes.values,
        "Is Null": df_complete.isnull().sum().values,
    }
)
df_time_analysis

,Column,Unique Values,Type,Is Null
0,Year,2,int64,0
1,Month,12,Int64,0
2,location_id,31,int64,0
3,crime_count,398,int64,0
4,weekday,371,int64,0
...,...,...,...,...
85,crime_count_ma6,580,float64,0
86,morning_ma6,437,float64,0
87,afternoon_ma6,482,float64,0
88,evening_ma6,405,float64,0


In [121]:
df_time_analysis["Column"].unique()

array(['Year', 'Month', 'location_id', 'crime_count', 'weekday',
       'weekend', 'morning', 'afternoon', 'evening', 'night',
       'holiday_num', 'num_days', 'morning_rate', 'afternoon_rate',
       'evening_rate', 'night_rate', 'sin_month', 'cos_month', 'time_id',
       'crime_pct_change', 'crime_pct_change_lag1', 'morning_rate_lag1',
       'afternoon_rate_lag1', 'evening_rate_lag1', 'night_rate_lag1',
       'crime_count_lag1', 'morning_lag1', 'afternoon_lag1',
       'evening_lag1', 'night_lag1', 'crime_pct_change_lag2',
       'morning_rate_lag2', 'afternoon_rate_lag2', 'evening_rate_lag2',
       'night_rate_lag2', 'crime_count_lag2', 'morning_lag2',
       'afternoon_lag2', 'evening_lag2', 'night_lag2',
       'crime_pct_change_lag3', 'morning_rate_lag3',
       'afternoon_rate_lag3', 'evening_rate_lag3', 'night_rate_lag3',
       'crime_count_lag3', 'morning_lag3', 'afternoon_lag3',
       'evening_lag3', 'night_lag3', 'crime_pct_change_lag6',
       'morning_rate_lag6', 'a

In [122]:
df_complete.to_csv(f"{data_path}time_series_data_AL.csv", index=False)